# Lab #3
In this laboratory, you will learn more about frequent itemsets and association rules.

## Onilne Retail Data set
The Online Retail Data Set [1] is a dataset made available on the UCI Machine Learning repository. It which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based online retail.

https://github.com/dbdmg/data-science-lab/raw/master/datasets/online_retail.csv

Each of the 541,909 rows contains an item that has been purchased by someone. Items can be grouped into invoices (you can think of these as receipts), where each invoice has been issued for a specific buyer, and can contain multiple items.
The columns contained in the CSV file are the following:
- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'C’, it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.

### Association rules from frequent itemses

**1 - Loading and cleaning the dataset**

Loading the dataset and removing invalid rows and those having an InvoiceNo that starts with "C", as they indicate that the invoice is about a cancelled purchase.

In [16]:
import pandas as pd
import csv

In [17]:
with open ("online_retail.csv") as f:
    reader = csv.reader(f)
    HretailDs=list(reader)
    
headers=HretailDs[0]
wholeRetailDs=HretailDs[1:]

In [18]:
#For debugging purpose
#print(headers)
#print(wholeRetailDs[:2])
#len(wholeRetailDs)

In [19]:
#da eliminare
import re

#Check if the string starts with "The" and ends with "Spain":

txts = ['NON Valido', 'VaLIDO',' 23 VALIDO ', '21 non valido',""," ? VF?",'VALIDO']
output=[]
for txt in txts:
    if not re.search("(?=.*[a-z])|^\\s*$|^\\?", txt.strip()): #or re.search("^\\s*$",txt):
        output.append(txt)
print(output)

[' 23 VALIDO ', 'VALIDO']


In [20]:
#Removing all the rows that has the invoice attribute that start with "C"
#Removingo other errors in the dataset
retailDs = []
for row in wholeRetailDs:
    if not row[0].startswith('C') and not re.search("(?=.*[a-z])|^\\s*$|^\\?", row[2].strip()):#tolgo tutti quelli che iniziano per C o che hanno delle minuscole
        retailDs.append(row)
len(retailDs)
#532621
#528519

528464

**2 - Aggregate at "invoice" level.**

Now that you have a dataset of items, you should aggregate it at an “invoice” level. For each invoice (identified by InvoiceNo) there can be multiple items (from multiple rows) in the dataset. For each invoice, you should build a list of all items belonging to it.

In [21]:
#Creating a dict to associate all the rows with the same InvoiceNo
tmpDict = {}
for row in retailDs:
    if (row[0] in tmpDict.keys()):
        tmpDict[row[0]].append(row[2].strip())
    else:
        tmpDict[row[0]]=[row[2].strip()]
invoices = list(tmpDict.values())
invoices[0]

['WHITE HANGING HEART T-LIGHT HOLDER',
 'WHITE METAL LANTERN',
 'CREAM CUPID HEARTS COAT HANGER',
 'KNITTED UNION FLAG HOT WATER BOTTLE',
 'RED WOOLLY HOTTIE WHITE HEART.',
 'SET 7 BABUSHKA NESTING BOXES',
 'GLASS STAR FROSTED T-LIGHT HOLDER']

In [22]:
#da eliminare
diz = {}
lProva = [[5,'a'],[5,'b'],[7,'z']]

for row in lProva:
    if (row[0] in diz.keys()):
        diz[row[0]].append(row[1])
    else:
        diz[row[0]]=[row[1]]
print(diz)
prova3=[["5",'a'],["5",' b'],[" 7",' z ','']]
llprova=[y.strip() for x in prova3 for y in x]
print(llprova)

{5: ['a', 'b'], 7: ['z']}
['5', 'a', '5', 'b', '7', 'z', '']


**3 - Converting the list of invoices into a matrix**

You should now have a list (one for each invoice) of lists (each list containing the items bought for that invoice). Now, we need to convert this into a matrix form. Of the many possible formats, we will use the one expected by the Mlxtend library, which is as follows. Given an ordered list of M possible items (in this case, all possible products that can be bought), and given N itemsets (in this case, invoices), we should build a matrix of N rows and M columns. The element at the ith row and jth column should be 1 if the $i^{th}$ itemset (invoice) contains the $j^{th}$ item (product), 0 otherwise.

In [23]:
#input: a list of invoices, each invoices is a list that contains a set of items
#output: a matrix of 0(item is absent) and 1(item is present)
def matrixation(invoices):
    linvoices=[x for l in invoices for x in l]
    sortedUniqueItemsList=sorted(list(set(linvoices)))
    matrix=[]
    for row in invoices:
        tmpL=[False]*len(sortedUniqueItemsList)
        for el in row:
            tmpL[sortedUniqueItemsList.index(el)]=True
        matrix.append(tmpL)
    return (sortedUniqueItemsList, matrix)

In [24]:
#Turing the matrix in a pandas DataFrame
all_items = matrixation(invoices)[0]
pa_matrix = matrixation(invoices)[1]
df = pd.DataFrame(data=pa_matrix, columns=all_items)

In [25]:
all_items

['10 COLOUR SPACEBOY PEN',
 '12 COLOURED PARTY BALLOONS',
 '12 DAISY PEGS IN WOOD BOX',
 '12 EGG HOUSE PAINTED WOOD',
 '12 HANGING EGGS HAND PAINTED',
 '12 IVORY ROSE PEG PLACE SETTINGS',
 '12 MESSAGE CARDS WITH ENVELOPES',
 '12 PENCIL SMALL TUBE WOODLAND',
 '12 PENCILS SMALL TUBE RED RETROSPOT',
 '12 PENCILS SMALL TUBE SKULL',
 '12 PENCILS TALL TUBE POSY',
 '12 PENCILS TALL TUBE RED RETROSPOT',
 '12 PENCILS TALL TUBE SKULLS',
 '12 PENCILS TALL TUBE WOODLAND',
 '12 PINK HEN+CHICKS IN BASKET',
 '12 PINK ROSE PEG PLACE SETTINGS',
 '12 RED ROSE PEG PLACE SETTINGS',
 '15 PINK FLUFFY CHICKS IN BOX',
 '15CM CHRISTMAS GLASS BALL 20 LIGHTS',
 '16 PC CUTLERY SET PANTRY DESIGN',
 '16 PIECE CUTLERY SET PANTRY DESIGN',
 '18PC WOODEN CUTLERY SET DISPOSABLE',
 '2 DAISIES HAIR COMB',
 '2 PICTURE BOOK EGGS EASTER BUNNY',
 '2 PICTURE BOOK EGGS EASTER CHICKS',
 '2 PICTURE BOOK EGGS EASTER DUCKS',
 '20 DOLLY PEGS RETROSPOT',
 '200 BENDY SKULL STRAWS',
 '200 RED + WHITE BENDY STRAWS',
 '20713',
 '3 BIRDS 

In [26]:
prova = [['a','b','c'],['b','c'],['a','c','d'],['a','b']]
print (prova)
matrixation(prova)

[['a', 'b', 'c'], ['b', 'c'], ['a', 'c', 'd'], ['a', 'b']]


(['a', 'b', 'c', 'd'],
 [[True, True, True, False],
  [False, True, True, False],
  [True, False, True, True],
  [True, True, False, False]])

**4 - Application of the *fp_growth* function**

With the df that you defined in the previous exercise, you can now use the fp_growth function. The first argument required is the previously built DataFrame, df. The second is the minimum support (minsup), i.e. the minimum fraction of the entire dataset in which the itemset should show up for it to be considered “frequent”. Try using different values of minsup, such as 0.5, 0.1, 0.05, 0.02, 0.01. How many results do you
obtain as minsup varies? 

In [27]:
from mlxtend.frequent_patterns import fpgrowth
fis=[]
for minsup in [0.5, 0.1, 0.05, 0.02, 0.01]:    
    fi = fpgrowth(df, minsup, use_colnames=True)
    print(len(fi))
    #print(fi.to_string())
    fis.append(fi)
#To get the corrisponding itemsets use "use_colnames=True"

0
3
34
382
1923


As the *minsup* lovers the number of frequent itemset increase

**5 - minsup = 0.02**

Consider the itemsets extracted for minsup = 0.02. How many items are contained? Which ones would you be considered the most useful?

In [28]:
fi = fis[3]
print(len(fi))
print(fi.to_string())

382
      support                                                                                            itemsets
0    0.113820                                                                (WHITE HANGING HEART T-LIGHT HOLDER)
1    0.023116                                                               (KNITTED UNION FLAG HOT WATER BOTTLE)
2    0.021606                                                                    (RED WOOLLY HOTTIE WHITE HEART.)
3    0.024124                                                                            (HAND WARMER UNION JACK)
4    0.073278                                                                     (ASSORTED COLOUR BIRD ORNAMENT)
5    0.039132                                                                          (HOME BUILDING BLOCK WORD)
6    0.031678                                                                          (LOVE BUILDING BLOCK WORD)
7    0.030218                                                                       

**6 - Association Rule Mining (*minsup* = 0.02)**

In [29]:
from mlxtend.frequent_patterns import association_rules
res = association_rules(fi, metric = "confidence", min_threshold=0.7)
res1 = res[["antecedents", "consequents", "support", "confidence", "lift"]]
res1
#if you want to filter on the confidence
#res2 = res1[res1["confidence"]>=0.7]

,antecedents,consequents,support,confidence,lift
0,"(JUMBO STORAGE BAG SUKI, JUMBO BAG PINK POLKADOT)",(JUMBO BAG RED RETROSPOT),0.020800,0.801942,7.611547
1,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.038678,0.756650,14.093853
2,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.038678,0.720450,14.093853
3,"(REGENCY CAKESTAND 3 TIER, GREEN REGENCY TEACU...",(ROSES REGENCY TEACUP AND SAUCER),0.020548,0.801572,14.930589
4,"(REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU...",(GREEN REGENCY TEACUP AND SAUCER),0.020548,0.777143,15.202905
5,(CHARLOTTE BAG PINK POLKADOT),(RED RETROSPOT CHARLOTTE BAG),0.026289,0.702557,13.491273
6,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.031880,0.826371,16.165929
7,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.030167,0.781984,14.565742
8,"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.027297,0.856240,15.948878
9,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.027297,0.904841,17.701016


**7 - Association rules (*minsup* = 0.01)**

In [31]:
fi = fis[4]
print(len(fi))
#print(fi.to_string())
res = association_rules(fi, metric = "confidence", min_threshold=0.85)
res1 = res[["antecedents", "consequents", "support", "confidence", "lift"]]
res1

1923


,antecedents,consequents,support,confidence,lift
0,"(JUMBO STORAGE BAG SUKI, JUMBO BAG BAROQUE BL...",(JUMBO BAG RED RETROSPOT),0.010375,0.872881,8.284862
1,"(STRAWBERRY CHARLOTTE BAG, CHARLOTTE BAG PINK ...",(RED RETROSPOT CHARLOTTE BAG),0.016418,0.860158,16.517702
2,"(CHARLOTTE BAG SUKI DESIGN, STRAWBERRY CHARLOT...",(RED RETROSPOT CHARLOTTE BAG),0.012339,0.904059,17.360731
3,"(STRAWBERRY CHARLOTTE BAG, WOODLAND CHARLOTTE ...",(RED RETROSPOT CHARLOTTE BAG),0.012188,0.920152,17.669768
4,"(STRAWBERRY CHARLOTTE BAG, WOODLAND CHARLOTTE ...",(RED RETROSPOT CHARLOTTE BAG),0.010274,0.918919,17.646087
5,"(CHARLOTTE BAG SUKI DESIGN, STRAWBERRY CHARLOT...",(RED RETROSPOT CHARLOTTE BAG),0.013094,0.858086,16.477903
6,"(JUMBO STORAGE BAG SUKI, JUMBO SHOPPER VINTAGE...",(JUMBO BAG RED RETROSPOT),0.013497,0.867314,8.232020
7,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.013346,0.895270,33.731473
8,(HERB MARKER BASIL),(HERB MARKER MINT),0.010475,0.859504,71.109642
9,(HERB MARKER MINT),(HERB MARKER BASIL),0.010475,0.866667,71.109642


### Apriori implementation

**1- Implement your own version of Apriori**


In [72]:
rows = [['a','b'],['b','c','d'],['a','c','d','e'],['a','d','e'],['a','b','c'],['a','b','c','d'],['b','c'],['a','b','c'],['a','b','d'],['b','c','e']]
allItems = sorted(list(set([el for l in rows for el in l])))
allItems

['a', 'b', 'c', 'd', 'e']

In [38]:
#Generate all items of lenght 1 and calculating its support
C1 = {}
for el in allItems:
    count = 0
    for l in rows:
        if el in l:
            count+=1
    C1[el]=count
C1

{'a': 7, 'b': 8, 'c': 7, 'd': 5, 'e': 3}

In [92]:
#input is a list of lists (each of them is a transaction)

def Apriori(rows, minsup):
    c=[] #Candidate frequent itemset of size k (in position k-1) (contains lists of dicts)
    L=[] #frequent itemset of size k (contains list of dict)
    
    tutti = sorted(list(set([el for l in rows for el in l])))
    diz = {}
    for el in tutti:
        diz[el] = None
    for el in diz.keys():
        count = 0
        for r in rows:
            if el in r:
                count += 1
        if count > minsup:
            diz[el] = count
    #elimino gli items con un valore minore di minsup
    filteredDiz = {k: v for k, v in diz.items() if v is not None}
    L.append(filteredDiz)
    print(L)
    
    while len(L[k])!= 0:
        #genero candidati a partire da l[k]
        c[k+1] = 
        k+=1
        


' while len(L[k])!= 0:\n        #genero candidati a partire da l[k]\n        c[k+1] = \n        k+=1'

In [93]:
l =[]
l.append({})
l[0]['a']=12
l

[{'a': 12}]

In [94]:
rows = [['a','b'],['b','c','d'],['a','c','d','e'],['a','d','e'],['a','b','c'],['a','b','c','d'],['b','c'],['a','b','c'],['a','b','d'],['b','c','e']]
Apriori(rows, 4)

[{'a': 7, 'b': 8, 'c': 7, 'd': 5}]
